In [1]:
from Net import Network
from Level import Delta
from LeastSquares import LS

h_level = Delta()
h_static = Delta("Static.txt")
#e_static = Delta("Static.txt", "Easting", "E")
#n_static = Delta("Static.txt", "Northing", "N")
h_def = Delta("Deformation.txt", "VD", "H")

models = [h_level, h_static, h_def]
#Height LSA
#remember to adjust to have the correct A matrix list and correct initial estimates
n = Network(models)

#final points
#n.x_0 + n.S + n.c[0]

#N and E LSA (linear only)
#remember to adjust to have the correct A matrix list and correct initial estimates
#models = [n_static, e_static]

#N, E, H nonlinear
#remember to adjust to have the correct A matrix list and correct initial estimates
#models = [h_level, h_static, h_def, n_static, e_static]
#n = Network(models = models)
#n.P 
#pd.DataFrame(np_array).to_csv("path/to/file.csv")
n.S + n.x_0

ASCM2 Could not be found
ASCM2 Could not be found
ASCM2 Could not be found
ASCM2 Could not be found


matrix([[-1110.04468284],
        [-1110.53380707],
        [-1110.53136463],
        [-1110.53927668],
        [-1110.53067516],
        [-1110.54694434],
        [-1110.52643795],
        [-1110.52782986],
        [-1110.26983179]])

In [2]:
from Direction import Horizontal_Direction
from Net import Network
from Level import Delta
from LeastSquares import LS
import math as m

d = Horizontal_Direction()
e_static = Delta("Static.txt", "Easting", "E")
n_static = Delta("Static.txt", "Northing", "N")

models = [e_static, n_static, d]
n = Network(models = models)

n.A.shape

C1 Could not be found
C1 Could not be found
C1 Could not be found
C1 Could not be found
C1 Could not be found
C1 Could not be found
C1 Could not be found
C1 Could not be found
C1 Could not be found
C1 Could not be found
C1 Could not be found
C1 Could not be found
C1 Could not be found
C1 Could not be found
C1 Could not be found
C1 Could not be found
C1 Could not be found
C1 Could not be found
C1 Could not be found
C1 Could not be found
C1 Could not be found
C1 Could not be found
C1 Could not be found
C1 Could not be found
C1 Could not be found
C1 Could not be found
C1 Could not be found
C1 Could not be found
C1 Could not be found
C1 Could not be found
C1 Could not be found
C1 Could not be found
1 Could not be found
1 Could not be found
1 Could not be found
1 Could not be found
C1 Could not be found
C1 Could not be found
C1 Could not be found
C1 Could not be found
C1 Could not be found
C1 Could not be found
C1 Could not be found
C1 Could not be found
C1 Could not be found
C1 Could not b

(87, 21)

In [9]:
n.S + n.x_0

matrix([[-3.64510080e+22],
        [-4.12722987e+22],
        [-3.65411096e+22],
        [-4.16044074e+22],
        [-3.02371255e+22],
        [-3.71715812e+22],
        [-2.21436442e+23],
        [-2.43941654e+23],
        [-6.93847545e+21],
        [-3.75207525e+22],
        [-2.58659811e+22],
        [-5.01029375e+22],
        [-2.44979712e+37],
        [-2.44979712e+37],
        [-3.44718530e+22],
        [-3.44718530e+22],
        [ 5.40271448e+06]])

In [2]:
n.c

matrix([[ 1.11023400e+03, -9.90378000e+03,  5.65909825e+06]])

In [3]:
n.x_0 + n.S

matrix([[ 6.10425507e+02],
        [ 1.26117399e+03],
        [ 2.55061929e+01],
        [ 5.23910890e+02],
        [ 1.32762864e+03],
        [ 2.51106354e+01],
        [-1.04176828e+01],
        [ 6.57257447e+02],
        [ 1.32256902e+03],
        [ 1.14372332e+00],
        [ 4.17799701e+02],
        [ 1.30443054e+03],
        [ 6.76324837e-01],
        [ 4.09495291e+02],
        [ 1.42031897e+03],
        [ 1.97405566e+00],
        [ 4.73248959e+02],
        [ 1.41707677e+03],
        [ 3.59856205e+00],
        [ 2.54311701e+01],
        [ 2.56171682e+01]])

In [6]:
n.S

matrix([[    9902.52050701],
        [-5659097.44300818],
        [    9902.39188983],
        [-5659097.59236266],
        [    9902.4134468 ],
        [-5659097.35198464],
        [    9902.42370126],
        [-5659097.78646438],
        [    9902.21429088],
        [-5659097.788033  ],
        [    9902.24595867],
        [-5659097.67522811]])

In [7]:
n.x_0 + n.S + n.c[0,2]

matrix([[5659708.67750701],
        [5660359.42599182],
        [5659622.16288983],
        [5660425.88063734],
        [5659755.5094468 ],
        [5660420.82101536],
        [5659516.05170126],
        [5660402.68253562],
        [5659507.74729089],
        [5660518.570967  ],
        [5659571.50095867],
        [5660515.32877189]])

In [7]:
n.c

matrix([[ 1.11023400e+03, -9.90378000e+03,  5.65909825e+06]])

In [8]:
n.A

matrix([[ 0.,  0.,  0., ...,  1.,  0.,  0.],
        [ 0.,  0.,  0., ..., -1.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        ...,
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [-1.,  0.,  0., ...,  0.,  0.,  0.]])

In [12]:
import numpy as np
import math as m

fro = np.array([-9484.624, 5660402.217])
to = np.array([-9492.719, 5660518.107])

north = np.array([0,1])

r_v = to - fro

np.dot(r_v / np.linalg.norm(r_v), north / np.linalg.norm(north))

57.15651235032544

In [5]:
np.array([1110.234)

array([1110.234])